### N Gram In N Gram Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 2

# # adding native word to shared word
# twogram_start = 0  # 0  # native word start index
# twogram_end = 200  # 28  # native word end index

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False
fourgram_sentence_check = False
fivegram_sentence_check = False

# n gram sample
threegram_sample = 6
fourgram_sample = 4
fivegram_sample = 2

# n gram select
twogram_select_start = 0
twogram_select_end = 10000

threegram_select_start = 0
threegram_select_end = 10000

fourgram_select_start = 0
fourgram_select_end = 10000

fivegram_select_start = 0
fivegram_select_end = 10000

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
    
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/5-N Gram In N Gram Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [6]:
def word_in_wordgroup_simple(df, source_column, target_column, word_sample_num):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [8]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [9]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [10]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [11]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [12]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

200

In [13]:
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram = df_twogram_sent.loc[:,["twogram","frequency"]]
    #df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]
else:
    df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")  
    df_twogram = df_twogram.loc[:,["twogram","frequency"]]
    #df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]

df_twogram

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
4457029,fikret cibran,3
4457030,romalı fikret,3
4457031,fikret ciooney,3
4457032,fikret cisco,3


In [14]:
d_list2  = df_twogram.iloc[:,0].values.tolist()

resultlist2 = []
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def word_in_wordgroup(d_list2):
    mergelist = []
    try:
        word = d_list2.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist2.append(d_list2)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list2) # string_word liste

result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=[0])  # add columns parameter for empty result
df_result2 = df_result2.rename(columns = {0: "twogram"})
df_merge2 = pd.merge(df_result2, df_twogram, how="left", on="twogram")
df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
df_merge_result2.drop_duplicates(inplace=True)
df_merge_result2.reset_index(drop=True, inplace=True)

# all word should be used at least once
#df_twogram_select = df_merge_result2.iloc[twogram_start:twogram_end,]

df_merge_result2

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
25473,yardım o,6
25474,doğru bize,6
25475,doğru bizim,6
25476,doğru biri,6


In [15]:
# word usage in twogram condition
word_num_dict = {}
for i in word_list:
    word_num_dict[f"{i}"] = 0

result_list_select = []
var_list = []
for i in range(len(df_merge_result2)):
    twogram = df_merge_result2.loc[i,"twogram"]
    frequency = df_merge_result2.loc[i,"frequency"]
    words = word_tokenize(twogram)   
    temp_list = [word for word in words]
    temp_list = temp_list + var_list
    # word count for max
    dict_list_count = Counter(temp_list)
    count_list = list(dict_list_count.values())
    # word count for min
    count_list2 = list(word_num_dict.values())

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        pass
    else:
        var_list = temp_list
        result_list_select.append([twogram,frequency]) 

        for item2 in dict_list_count.items(): 
            word_num_dict[item2[0]] = item2[1]        

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}")

Max condition: False 
Min condition: False


In [16]:
#word_num_dict

In [17]:
#Counter(var_list)

In [18]:
df_twogram_select = pd.DataFrame(result_list_select, columns=["twogram","frequency"])
df_twogram_select.sort_values(by="frequency", ascending=False, inplace=True)
df_twogram_select.reset_index(drop=True, inplace=True)
df_twogram_select

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
193,bence kendi,154
194,yine yapıyorsun,135
195,zaten tüm,86
196,saat hala,57


In [19]:
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram = df_threegram_sent.loc[:,["threegram","frequency"]]
else:
    df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")  
    df_threegram = df_threegram.loc[:,["threegram","frequency"]]

df_threegram

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [20]:
d_list3  = df_threegram.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,başka bir şey,75193
4,hayır hayır hayır,69443
...,...,...
109758,gece onu neden,10
109759,gece onu nasıl,10
109760,bu küçük de,10
109761,gece olur ve,10


In [21]:
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fourgram = df_fourgram_sent.loc[:,["fourgram","frequency"]]
else:
    df_fourgram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")  
    df_fourgram = df_fourgram.loc[:,["fourgram","frequency"]]

df_fourgram

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
3052209,fikret miloya merhaba de,5
3052210,fikret millsin oğlu mu,5
3052211,fikret millet iyi iş,5
3052212,fikret millet bize bakıyor,5


In [22]:
d_list4  = df_fourgram.iloc[:,0].values.tolist()

resultlist4 = []
manager = multiprocessing.Manager()
resultlist4 = manager.list()

def word_in_wordgroup(d_list4):
    mergelist = []
    try:
        word = d_list4.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist4.append(d_list4)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list4) # string_word liste

result_list4 = list(resultlist4)
df_result4 = pd.DataFrame(result_list4, columns=[0])  # add columns parameter for empty result
df_result4 = df_result4.rename(columns = {0: "fourgram"})
df_merge4 = pd.merge(df_result4, df_fourgram, how="left", on="fourgram")
df_merge_result4 = df_merge4.sort_values(by="frequency", ascending=False)
df_merge_result4.drop_duplicates(inplace=True)
df_merge_result4.reset_index(drop=True, inplace=True)
df_fourgram_select = df_merge_result4
df_fourgram_select

,fourgram,frequency
0,bir şey var mı,41773
1,bir sorun mu var,21423
2,bu da ne demek,16884
3,en son ne zaman,15141
4,başka bir şey var,12619
...,...,...
79278,bilmiyorum ama bana onun,7
79279,ne demek bir adam,7
79280,bilmiyorum ama ama bu,7
79281,bilmiyorum ama artık bu,7


In [23]:
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fivegram = df_fivegram_sent.loc[:,["fivegram","frequency"]]
else:
    df_fivegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")  
    df_fivegram = df_fivegram.loc[:,["fivegram","frequency"]]

df_fivegram

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4


In [24]:
d_list5  = df_fivegram.iloc[:,0].values.tolist()

resultlist5 = []
manager = multiprocessing.Manager()
resultlist5 = manager.list()

def word_in_wordgroup(d_list5):
    mergelist = []
    try:
        word = d_list5.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist5.append(d_list5)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list5) # string_word liste

result_list5 = list(resultlist5)
df_result5 = pd.DataFrame(result_list5, columns=[0])  # add columns parameter for empty result
df_result5 = df_result5.rename(columns = {0: "fivegram"})
df_merge5 = pd.merge(df_result5, df_fivegram, how="left", on="fivegram")
df_merge_result5 = df_merge5.sort_values(by="frequency", ascending=False)
df_merge_result5.drop_duplicates(inplace=True)
df_merge_result5.reset_index(drop=True, inplace=True)
df_fivegram_select = df_merge_result5
df_fivegram_select

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,bu da ne demek şimdi,4791
4,hayır hayır hayır hayır hayır,4380
...,...,...
16040,biri değilim ama ona ne,10
16041,çok yeni biliyorum ama öyle,10
16042,ama burada hiç para yok,10
16043,bak bir şey daha var,10


In [25]:
df_ngram_concat = pd.concat([df_twogram_select, df_threegram_select, df_fourgram_select, df_fivegram_select], axis=1)
df_ngram_concat

,twogram,frequency,threegram,frequency,fourgram,frequency,fivegram,frequency
0,bir şey,859944.0,bir şey yok,113165,bir şey var mı,41773.0,başka bir şey var mı,14104.0
1,değil mi,585879.0,bir şey var,110455,bir sorun mu var,21423.0,bu da ne demek oluyor,10205.0
2,ben de,377765.0,bu da ne,89463,bu da ne demek,16884.0,o kadar da kötü değil,7012.0
3,teşekkür ederim,370619.0,başka bir şey,75193,en son ne zaman,15141.0,bu da ne demek şimdi,4791.0
4,ne oldu,322758.0,hayır hayır hayır,69443,başka bir şey var,12619.0,hayır hayır hayır hayır hayır,4380.0
...,...,...,...,...,...,...,...,...
109758,NaN,NaN,gece onu neden,10,NaN,NaN,NaN,NaN
109759,NaN,NaN,gece onu nasıl,10,NaN,NaN,NaN,NaN
109760,NaN,NaN,bu küçük de,10,NaN,NaN,NaN,NaN
109761,NaN,NaN,gece olur ve,10,NaN,NaN,NaN,NaN


In [26]:
df_two_in_three = word_in_wordgroup_simple(df_ngram_concat, "twogram","threegram",threegram_sample)
df_two_in_four = word_in_wordgroup_simple(df_ngram_concat, "twogram","fourgram",fourgram_sample)
df_two_in_five = word_in_wordgroup_simple(df_ngram_concat, "twogram","fivegram",fivegram_sample)

In [27]:
df_twogram_order_join_threegram = df_two_in_three.groupby(["twogram"])["threegram"].apply(", ".join).reset_index()
df_twogram_order_join_fourgram = df_two_in_four.groupby(["twogram"])["fourgram"].apply(", ".join).reset_index()
df_twogram_order_join_fivegram = df_two_in_five.groupby(["twogram"])["fivegram"].apply(", ".join).reset_index()  

In [28]:
dfs = [df_twogram_order_join_threegram, df_twogram_order_join_fourgram, df_twogram_order_join_fivegram]

In [29]:
df_ngram_sample_join_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='outer'), dfs)
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge 

,twogram,threegram,fourgram,fivegram
0,adam vardı,"bir adam vardı, adam vardı ya, adam vardı ve, ...","bir adam vardı ve, olan bir adam vardı, diye b...","daha önce bir adam vardı, diye bir adam vardı ya"
1,al bakalım,"al bakalım dostum, tamam al bakalım, de al bak...","gel de al bakalım, al bakalım bu senin, mı ist...","al bakalım bu senin için, hayır o değil al bak..."
2,al şunu,"hadi al şunu, al şunu ve, al şunu al, al şunu ...","al şunu al şunu, al şunu ve git, al şunu da gi...","hadi al şunu ve git, gel buraya ve al şunu"
3,ama şimdi,"ama şimdi değil, ama şimdi bir, evet ama şimdi...","ama şimdi her şey, ama şimdi daha iyi, ama şim...","ama şimdi doğru olduğunu biliyorum, ama şimdi ..."
4,artık hiçbir,"artık hiçbir şey, artık hiçbir şeyi, ama artık...","artık hiçbir şey bilmiyorum, artık hiçbir şey ...","bilmiyorum bana artık hiçbir şey, artık hiçbir..."
...,...,...,...,...
191,öyle mi,"oh öyle mi, gerçekten öyle mi, ya öyle mi, yan...","öyle mi değil mi, sen de öyle mi, şey yok öyle...","bir şey yok öyle mi, burada olduğunu bilmiyoru..."
192,üzgünüm özür,"çok üzgünüm özür, oh üzgünüm özür",üzgünüm çok üzgünüm özür,NaN
193,üç iki,"üç iki bir, ve üç iki, üç iki ve, son üç iki, ...","üç iki bir şimdi, üç iki bir ve, ve üç iki bir...",NaN
194,şimdi olmaz,"hayır şimdi olmaz, ama şimdi olmaz, şimdi olma...","şimdi olmaz tamam mı, hayır şimdi olmaz şimdi,...",hayır şimdi olmaz şimdi olmaz


In [30]:
df_ngram_sample_join_merge = pd.merge(df_ngram_sample_join_merge, df_twogram_select, how="right", on="twogram")
df_ngram_sample_join_merge.drop_duplicates(inplace=True)
df_ngram_sample_join_merge.rename(columns={"frequency":"two_freq"}, inplace=True)
df_ngram_sample_join_merge.sort_values(by="two_freq", ascending=False, inplace=True)
df_ngram_sample_join_merge.reset_index(drop=True, inplace=True)
df_ngram_sample_join_merge

,twogram,threegram,fourgram,fivegram,two_freq
0,bir şey,"bir şey yok, bir şey var, başka bir şey, bir ş...","bir şey var mı, başka bir şey var, bir şey dah...","başka bir şey var mı, daha önce böyle bir şey",859944
1,değil mi,"öyle değil mi, biliyorsun değil mi, güzel deği...","de öyle değil mi, bir şey değil mi, çok güzel ...","ne kadar güzel değil mi, güzel bir gün değil mi",585879
2,ben de,"ben de seni, ben de öyle, ve ben de, evet ben ...","o zaman ben de, ben de aynı şeyi, evet ben de ...","sen de biliyorsun ben de, ben de sana aynı şeyi",377765
3,teşekkür ederim,"çok teşekkür ederim, için teşekkür ederim, teş...","için çok teşekkür ederim, şey için teşekkür ed...","her şey için teşekkür ederim, teşekkür ederim ...",370619
4,ne oldu,"sonra ne oldu, ne oldu sana, sana ne oldu, ne ...","sana ne oldu böyle, ne oldu sana böyle, ne old...","sonra ne oldu biliyor musun, ve ne oldu biliyo...",322758
...,...,...,...,...,...
193,bence kendi,ama bence kendi,NaN,NaN,154
194,yine yapıyorsun,"şimdi yine yapıyorsun, işte yine yapıyorsun",NaN,NaN,135
195,zaten tüm,"zaten tüm bu, sen zaten tüm, zaten tüm gün",sen zaten tüm gün,NaN,86
196,saat hala,son saat hala,NaN,NaN,57


In [31]:
df_ngram_sample_join_merge.to_excel(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_With_{word_end}_Word_Join_Result.xlsx", index=False)

#### Adding Frequency For Ratio

In [32]:
df_two_in_three_freq = pd.merge(df_two_in_three, df_threegram, how="left", on="threegram")
df_two_in_three_freq.drop_duplicates(inplace=True)
df_two_in_three_freq.rename(columns={"frequency":"three_freq"}, inplace=True)
df_two_in_three_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_three_freq

,threegram,three_freq
0,bir şey yok,113165
1,bir şey var,110455
2,başka bir şey,75193
3,bir şey değil,68498
4,bir şey mi,64933
...,...,...
1119,işte yine yapıyorsun,12
1120,zaten tüm bu,32
1121,sen zaten tüm,14
1122,zaten tüm gün,14


In [33]:
df_two_in_four_freq = pd.merge(df_two_in_four, df_fourgram, how="left", on="fourgram")
df_two_in_four_freq.drop_duplicates(inplace=True)
df_two_in_four_freq.rename(columns={"frequency":"four_freq"}, inplace=True)
df_two_in_four_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_four_freq

,fourgram,four_freq
0,bir şey var mı,41773
1,başka bir şey var,12619
2,bir şey daha var,12185
3,bir şey mi var,11834
4,de öyle değil mi,5379
...,...,...
701,ben aslında gerçek bir,12
702,da aslında gerçek bir,11
703,o aslında gerçek bir,7
704,o adam yine geldi,11


In [34]:
df_two_in_five_freq = pd.merge(df_two_in_five, df_fivegram, how="left", on="fivegram")
df_two_in_five_freq.drop_duplicates(inplace=True)
df_two_in_five_freq.rename(columns={"frequency":"five_freq"}, inplace=True)
df_two_in_five_freq.drop(["twogram"], axis=1, inplace=True)
df_two_in_five_freq

,fivegram,five_freq
0,başka bir şey var mı,14104
1,daha önce böyle bir şey,2160
2,ne kadar güzel değil mi,1647
3,güzel bir gün değil mi,1423
4,sen de biliyorsun ben de,296
...,...,...
293,şunu bu sana lazım olacak,11
294,belki de bu yüzden onları,12
295,ben de bu yüzden sizi,17
296,sanırım bu yüzden sizi bize,13


In [35]:
df_ngram_sample_concat = pd.concat([df_twogram_select,df_two_in_three_freq, df_two_in_four_freq, df_two_in_five_freq], axis=1)
df_ngram_sample_concat

,twogram,frequency,threegram,three_freq,fourgram,four_freq,fivegram,five_freq
0,bir şey,859944.0,bir şey yok,113165,bir şey var mı,41773.0,başka bir şey var mı,14104.0
1,değil mi,585879.0,bir şey var,110455,başka bir şey var,12619.0,daha önce böyle bir şey,2160.0
2,ben de,377765.0,başka bir şey,75193,bir şey daha var,12185.0,ne kadar güzel değil mi,1647.0
3,teşekkür ederim,370619.0,bir şey değil,68498,bir şey mi var,11834.0,güzel bir gün değil mi,1423.0
4,ne oldu,322758.0,bir şey mi,64933,de öyle değil mi,5379.0,sen de biliyorsun ben de,296.0
...,...,...,...,...,...,...,...,...
1119,NaN,NaN,işte yine yapıyorsun,12,NaN,NaN,NaN,NaN
1120,NaN,NaN,zaten tüm bu,32,NaN,NaN,NaN,NaN
1121,NaN,NaN,sen zaten tüm,14,NaN,NaN,NaN,NaN
1122,NaN,NaN,zaten tüm gün,14,NaN,NaN,NaN,NaN


In [36]:
df_ngram_sample_concat.to_excel(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_With_{word_end}_Word_Frequency_Result.xlsx", index=False)

#### Calculate Result And Select Ratio

In [37]:
df_ngram_threegram_unique = df_ngram_sample_concat[["threegram","three_freq"]].drop_duplicates()
df_ngram_fourgram_unique = df_ngram_sample_concat[["fourgram","four_freq"]].drop_duplicates()
df_ngram_fivegram_unique = df_ngram_sample_concat[["fivegram","five_freq"]].drop_duplicates()

In [38]:
twogram_result_freq = df_twogram_select["frequency"].sum()
threegram_result_freq = df_ngram_threegram_unique["three_freq"].sum()
fourgram_result_freq = df_ngram_fourgram_unique["four_freq"].sum()
fivegram_result_freq = df_ngram_fivegram_unique["five_freq"].sum()

In [39]:
twogram_select_freq = df_twogram.iloc[twogram_select_start:twogram_select_end,]["frequency"].sum()
threegram_select_freq = df_threegram.iloc[threegram_select_start:threegram_select_end,]["frequency"].sum()
fourgram_select_freq = df_fourgram.iloc[fourgram_select_start:fourgram_select_end,]["frequency"].sum()
fivegram_select_freq = df_fivegram.iloc[fivegram_select_start:fivegram_select_end,]["frequency"].sum()

In [40]:
(twogram_result_freq/twogram_select_freq)*100

14.315826403872556

In [41]:
(threegram_result_freq/threegram_select_freq)*100

14.698425038102114

In [42]:
(fourgram_result_freq/fourgram_select_freq)*100

9.316559114531904

In [43]:
(fivegram_result_freq/fivegram_select_freq)*100

5.061893379008326

#### Copy Move And Delete

In [44]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{len(df_twogram_select)}_Twogram_In_{threegram_sample}_Threegram_{fourgram_sample}_\
Fourgram_{fivegram_sample}_Fivegram_Sample_*_Result.xlsx")
output_file

['Turkish_198_Twogram_In_6_Threegram_4_Fourgram_2_Fivegram_Sample_With_200_Word_Join_Result.xlsx',
 'Turkish_198_Twogram_In_6_Threegram_4_Fourgram_2_Fivegram_Sample_With_200_Word_Frequency_Result.xlsx']

In [45]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [46]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass